In [5]:
import torch
import numpy as np
import pyro
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS

import pandas as pd
from matplotlib import pyplot as plt
import random
import models
import gdown

In [15]:
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTltw5xQVB_sSBCLiA5nzbiDc1srkInw5TDBWcYy50A-Yhr7zKMTgJUy0aoE1q0uCo5WUJSJSVhR_SY/pub?gid=0&single=true&output=csv"
df = pd.read_csv(url)

In [16]:
#file_path = 'results.ods'

# Read the Excel file
#df = pd.read_excel(file_path, engine="odf")

# Print the DataFrame
print(df)

winners=np.array(df['Winner'])
losers=df['Loser']
loser_scores=df['Points']
names = pd.concat([df['Winner'], df['Loser']]).unique()
ids = list(range(len(names)))
dict_to_id = dict(zip(names,ids))
dict_to_name=dict(zip(ids,names))
winner_ids = [dict_to_id[i] for i in winners]
loser_ids = [dict_to_id[i] for i in losers]

    Winner    Loser  Points
0     Nour     Fedo       2
1     Nour    Mauro       8
2    Mauro  Ma Long       7
3  Ma Long  Waldner       8
4  Waldner  Ma Long       7
5     Boll     Fedo       3
6    Mauro     Boll       5
7    Mauro     Nour      10
8  Waldner     Boll       9
9  Waldner     Fedo       5


In [17]:
winner_ids=torch.Tensor(winner_ids).int()
loser_ids=torch.Tensor(loser_ids).int()
loser_scores=torch.Tensor(loser_scores).int()

In [20]:
# Run inference using NUTS (No-U-Turn Sampler) in MCMC
nuts_kernel = NUTS(models.pyro_model)
mcmc_run = MCMC(nuts_kernel, num_samples=1000, warmup_steps=100)
mcmc_run.run(winner_ids, loser_ids, loser_scores)

# Get posterior samples
posterior_samples = mcmc_run.get_samples()




Sample: 100%|█| 1100/1100 [00:26, 41.25it/s, step size=2.33e-01, acc. prob=0.915


In [21]:
# Print the posterior mean and standard deviation of player strengths
for i in range(len(ids)):
    player = f'Player {i+1}: {dict_to_name[i]}'
    print(player)
    print("Posterior mean of strength:", posterior_samples['strength'][:, i].mean().item())
    print("Posterior std dev of strength:", posterior_samples['strength'][:, i].std().item())
    print()

Player 1: Nour
Posterior mean of strength: -1.0182446241378784
Posterior std dev of strength: 0.39145851135253906

Player 2: Mauro
Posterior mean of strength: -0.9551647901535034
Posterior std dev of strength: 0.3584935963153839

Player 3: Ma Long
Posterior mean of strength: -0.6248592734336853
Posterior std dev of strength: 0.2524974048137665

Player 4: Waldner
Posterior mean of strength: -0.639363169670105
Posterior std dev of strength: 0.2573464810848236

Player 5: Boll
Posterior mean of strength: -0.5731425285339355
Posterior std dev of strength: 0.2498534917831421

Player 6: Fedo
Posterior mean of strength: -0.23513329029083252
Posterior std dev of strength: 0.10877437889575958

